In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/sms-spam-collection-dataset/spam.csv


In [23]:
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
sms = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='latin')

In [12]:
print(sms.head())

                                                            labels menssagens
v1   v2                                                 NaN    NaN        NaN
ham  Go until jurong point, crazy.. Available only i... NaN    NaN        NaN
     Ok lar... Joking wif u oni...                      NaN    NaN        NaN
spam Free entry in 2 a wkly comp to win FA Cup final... NaN    NaN        NaN
ham  U dun say so early hor... U c already then say...  NaN    NaN        NaN


In [14]:
# Removendo as colunas desnecessárias
sms = sms.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)

In [19]:
print(sms.head())

  label                                         menssagens  tamanho
0   ham  Go until jurong point, crazy.. Available only ...      111
1   ham                      Ok lar... Joking wif u oni...       29
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...      155
3   ham  U dun say so early hor... U c already then say...       49
4   ham  Nah I don't think he goes to usf, he lives aro...       61


In [16]:
sms = sms.rename(columns = {'v1':'label','v2':'menssagens'})

In [18]:
sms['tamanho'] = sms['menssagens'].apply(len)

In [ ]:
# Convertendo as strings em valores numericos

In [22]:
# Retira as pontuações e palavras de parada (stop words)
def text_process(mess):
    nopunc =[char for char in mess if char not in string.punctuation]# Retira a pontuação
    nopunc=''.join(nopunc)
    # Retira a pontuação
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [25]:
# Tokenizando, isto é, retirando as palavras que importam
# Contando quantas vezes cada palavra apareceu
bow_transformer = CountVectorizer(analyzer=text_process).fit(sms['menssagens'])

In [26]:
# Transformando todo o data frame ...
messages_bow = bow_transformer.transform(sms['menssagens'])

In [ ]:
# TF-IDF significa termo frequência de documento inversa à frequência
# TF (t) = (número de vezes que o termo t aparece em um documento) / (número total de termos no documento).
# IDF (t) = log_e (Número total de documentos / Número de documentos com o termo t).

In [27]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer().fit(messages_bow)

In [28]:
messages_tfidf=tfidf_transformer.transform(messages_bow)

In [29]:
# Treinando o modelo
from sklearn.naive_bayes import GaussianNB

In [30]:
gnb = GaussianNB()

In [35]:
pred = gnb.fit(messages_tfidf.toarray(), sms['label'])

In [53]:
# Teste
def transformer(msg):
    teste = bow_transformer.transform([msg])
    return teste.toarray()

In [54]:
print('Valor predito: ', pred.predict(teste.toarray())[0])
print('Valor esperado: ', sms['label'][3])

Valor predito:  ham
Valor esperado:  ham


In [58]:
count = 0
for i in range(0, 5572):
    if( pred.predict([transformer(sms['menssagens'][i])[0]]) != sms['label'][i]):
        count = count + 1
        #print(sms['label'][i])
        #print(sms['menssagens'][i])
        
print(f'O classificador errou: {count} vezes, resultando em {(count * 100)/len(sms)}% de erro')

O classificador errou: 167 vezes, resultando em 2.9971284996410623%
